# 精度を意識したモデルの作成
機械学習を使って何らかの分析をする場合、基本的に汎化した際のモデルの精度が一つの信頼の指標になる事がある。ここでは、よく使用される最適なモデルの作成方法から力業を紹介する。

本項では、SVMについて精度の高いモデルを作成する例を紹介する。

## ライブラリのインポート

In [1]:
from sklearn.model_selection import train_test_split as tts
from sklearn.model_selection import GridSearchCV as gcv
from sklearn.model_selection import cross_val_score as cvs
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
import scipy.stats as stats
import pandas as pd
import numpy as np

## データの読み込み

In [2]:
df = pd.read_csv("wine.csv")
df.head()

,Wine,Alcohol,Malic.acid,Ash,Acl,Mg,Phenols,Flavanoids,Nonflavanoid.phenols,Proanth,Color.int,Hue,OD,Proline
0,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
1,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
2,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
3,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
4,1,14.20,1.76,2.45,15.2,112,3.27,3.39,0.34,1.97,6.75,1.05,2.85,1450


In [3]:
y_name = "Wine"
y = df[y_name].values
x_tab = df.drop([y_name], axis=1)
x_name = x_tab.columns
x = x_tab.values

In [4]:
x_train, x_test, y_train, y_test = tts(x, y, test_size=0.3, random_state=0)

## グリッドサーチ
簡単な機械学習のモデルであればパラメータを指定する必要がないが、機械学習のモデルによってはパラメータを指定してモデルの設計についてを調節することで精度を向上させる。ここでは、SVMのカーネル関数、コスト、ガンマ値についての最適な数値を探索する。

In [5]:
param = {
    "kernel":["linear", "rbf", "poly", "sigmoid"],
    "C":[10, 1, 0.1],
    "gamma":[2**-10, 2, 2**5]
}

In [6]:
model = gcv(SVC(), param, cv=5, verbose=2)
model.fit(x_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV] END ............C=10, gamma=0.0009765625, kernel=linear; total time=   0.0s
[CV] END ............C=10, gamma=0.0009765625, kernel=linear; total time=   0.0s
[CV] END ............C=10, gamma=0.0009765625, kernel=linear; total time=   0.0s
[CV] END ............C=10, gamma=0.0009765625, kernel=linear; total time=   0.0s
[CV] END ............C=10, gamma=0.0009765625, kernel=linear; total time=   0.0s
[CV] END ...............C=10, gamma=0.0009765625, kernel=rbf; total time=   0.0s
[CV] END ...............C=10, gamma=0.0009765625, kernel=rbf; total time=   0.0s
[CV] END ...............C=10, gamma=0.0009765625, kernel=rbf; total time=   0.0s
[CV] END ...............C=10, gamma=0.0009765625, kernel=rbf; total time=   0.0s
[CV] END ...............C=10, gamma=0.0009765625, kernel=rbf; total time=   0.0s
[CV] END ..............C=10, gamma=0.0009765625, kernel=poly; total time=   0.1s
[CV] END ..............C=10, gamma=0.0009765625

[CV] END .......................C=1, gamma=32, kernel=linear; total time=   0.0s
[CV] END .......................C=1, gamma=32, kernel=linear; total time=   0.0s
[CV] END .......................C=1, gamma=32, kernel=linear; total time=   0.0s
[CV] END ..........................C=1, gamma=32, kernel=rbf; total time=   0.0s
[CV] END ..........................C=1, gamma=32, kernel=rbf; total time=   0.0s
[CV] END ..........................C=1, gamma=32, kernel=rbf; total time=   0.0s
[CV] END ..........................C=1, gamma=32, kernel=rbf; total time=   0.0s
[CV] END ..........................C=1, gamma=32, kernel=rbf; total time=   0.0s
[CV] END .........................C=1, gamma=32, kernel=poly; total time=   0.1s
[CV] END .........................C=1, gamma=32, kernel=poly; total time=   0.0s
[CV] END .........................C=1, gamma=32, kernel=poly; total time=   0.0s
[CV] END .........................C=1, gamma=32, kernel=poly; total time=   0.0s
[CV] END ...................

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [10, 1, 0.1], 'gamma': [0.0009765625, 2, 32],
                         'kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
             verbose=2)

In [7]:
model.best_estimator_

SVC(C=10, gamma=0.0009765625, kernel='linear')

In [8]:
model = SVC(C=10, gamma=0.0009765625, kernel="linear")
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print("Accuracy Score: %.2f"%(accuracy_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))

Accuracy Score: 0.93
              precision    recall  f1-score   support

           1       0.95      0.95      0.95        19
           2       0.90      0.90      0.90        21
           3       0.93      0.93      0.93        14

    accuracy                           0.93        54
   macro avg       0.93      0.93      0.93        54
weighted avg       0.93      0.93      0.93        54



## 交差検証
交差検証では、データセットについて一部を検証データとして他のデータを学習させて検証データでテストするといった流れを全データで巡回する。

In [9]:
model = SVC(C=10, gamma=0.0009765625, kernel="linear")
score = cvs(model, x_train, y_train, cv=5)
score

array([0.96      , 0.92      , 0.96      , 0.96      , 0.91666667])

In [10]:
print("  mode:%.2f"%(stats.mode(score)[0]))
print("median:%.2f"%(np.median(score)))
print("  mean:%.2f"%(np.mean(score)))

  mode:0.96
median:0.96
  mean:0.94


## 検証用データの使用
本節は非常に力業になるが、あらかじめ検討するモデルを複数作成する。ここで元々の訓練データをさらに訓練データと検証データに毎回ランダムで分けて各モデルに学習させ、モデルと精度(ここでは正解率)をリストに追加していく。最後にリストを精度で降順にして最も精度の高いモデルを選択する。

In [11]:
models = []
for i in range(50):
    x_train2, x_val, y_train2, y_val = tts(x_train, y_train, test_size=0.2, random_state=i)
    model1 = SVC(kernel="rbf", C=0.1)
    model2 = SVC(kernel="linear", C=0.1)
    model3 = SVC(kernel="poly", C=0.1)
    model4 = SVC(kernel="rbf", C=10)
    model5 = SVC(kernel="linear", C=10)
    model6 = SVC(kernel="poly", C=10)
    model1.fit(x_train2, y_train2)
    model2.fit(x_train2, y_train2)
    model3.fit(x_train2, y_train2)
    model4.fit(x_train2, y_train2)
    model5.fit(x_train2, y_train2)
    model6.fit(x_train2, y_train2)
    y_pred1 = model1.predict(x_val)
    y_pred2 = model2.predict(x_val)
    y_pred3 = model3.predict(x_val)
    y_pred4 = model4.predict(x_val)
    y_pred5 = model5.predict(x_val)
    y_pred6 = model6.predict(x_val)
    models.append([model1, accuracy_score(y_val, y_pred1)])
    models.append([model2, accuracy_score(y_val, y_pred2)])
    models.append([model3, accuracy_score(y_val, y_pred3)])
    models.append([model4, accuracy_score(y_val, y_pred4)])
    models.append([model5, accuracy_score(y_val, y_pred5)])
    models.append([model6, accuracy_score(y_val, y_pred6)])

In [12]:
models = sorted(models, key=lambda x: x[1], reverse=True)

In [13]:
model = models[0][0]
y_pred = model.predict(x_test)
print("Accuracy Score: %.2f"%(accuracy_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))

Accuracy Score: 0.96
              precision    recall  f1-score   support

           1       0.95      0.95      0.95        19
           2       0.95      0.95      0.95        21
           3       1.00      1.00      1.00        14

    accuracy                           0.96        54
   macro avg       0.97      0.97      0.97        54
weighted avg       0.96      0.96      0.96        54



## 変数選択
ここまで、学習のさせ方やパラメータチューニングについてを行ったが、今度は使用する変数を選択して分類に寄与する変数を使用してモデルを作成する。ここでは分類寄与率(重要度)を累積して9割になる変数までを学習に使う。

In [14]:
tree = DTC()
tree.fit(x_train, y_train)
imp = tree.feature_importances_
dfi = pd.DataFrame(imp)
dfi.index = x_name
dfi.columns = ["重要度"]
dfi = dfi.sort_values("重要度", ascending=False)
dfi

,重要度
Flavanoids,0.403550
Color.int,0.381003
Proline,0.123415
OD,0.046784
Hue,0.023806
Alcohol,0.021443
Malic.acid,0.000000
Ash,0.000000
Acl,0.000000
Mg,0.000000


In [15]:
drop_col = ["Acl", "Alcohol", "Malic.acid", "Ash", "Mg", "Phenols", "Nonflavanoid.phenols", "Proanth", "Hue"]
x_tab2 = x_tab.drop(drop_col, axis=1)
x2 = x_tab2.values
x2_train, x2_test, y2_train, y2_test = tts(x2, y, test_size=0.3, random_state=0)

In [16]:
model = SVC(C=10, gamma=0.0009765625, kernel="linear")
model.fit(x2_train, y2_train)
y_pred = model.predict(x2_test)
print("Accuracy Score: %.2f"%(accuracy_score(y2_test, y_pred)))
print(classification_report(y2_test, y_pred))

Accuracy Score: 0.98
              precision    recall  f1-score   support

           1       1.00      0.95      0.97        19
           2       0.95      1.00      0.98        21
           3       1.00      1.00      1.00        14

    accuracy                           0.98        54
   macro avg       0.98      0.98      0.98        54
weighted avg       0.98      0.98      0.98        54



ここで、過去の精度と比較してみると、パラメータのチューニングや学習に使用するデータだけでなく変数選択も行うことで、より精度を向上させられることが分かる。